In [1]:
from dwave.system import LeapHybridSampler,DWaveSampler, EmbeddingComposite
from collections import defaultdict
import math

This program solves the 9x9 Sudoku problem using D-Waves quantum computer

In [2]:
sampler = LeapHybridSampler()

In [3]:

S=[
    [0,0,5,3,0,0,0,0,0],
    [8,0,0,0,0,0,0,2,0],
    [0,7,0,0,0,0,5,0,0],
    [4,0,0,0,0,5,3,0,0],
    [0,1,0,0,7,0,0,0,6],
    [0,0,3,2,0,0,0,8,0],
    [0,6,0,5,0,0,0,0,1],
    [0,0,4,0,0,0,0,3,0],
    [0,0,0,0,0,9,7,0,0]
]

n = len(S)
N=int(math.sqrt(n))
rn = range(n)

def f(i: int,j: int,k: int) -> int:
    return "{}{}{}".format(i,j,k)

In [4]:
Box=list()
InBox=list()
for i in range(N):
    for j in range(N):
        Box.append((i*N,j*N))
        InBox.append((i,j))
print(Box,InBox)

[(0, 0), (0, 3), (0, 6), (3, 0), (3, 3), (3, 6), (6, 0), (6, 3), (6, 6)] [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]


In [5]:
Q = defaultdict(int)
c=1
        
for i in rn:
    for j in rn:
        for k in rn:
            
            cel1 = f(i,j,k)
            row1 = f(j,k,i)
            col1 = f(k,j,i)
            
            # Hints
            if S[i][j] == k+1:
                Q[(cel1,cel1)]   = -2*c
            else:
                Q[(cel1,cel1)]  = -c

            
            
            for k2 in range(k+1,n):
                
                cel2 = f(i,j,k2)
                row2 = f(j,k2,i)
                col2 = f(k2,j,i)
                
                Q[(cel1,cel2)]  = 2*c
                Q[(row1,row2)]  = 2*c
                Q[(col1,col2)]  = 2*c
            
            # Box number
            i0 = (j // N)*N
            j0 = (k // N)*N
            # Boxes
            for k1 in rn:
                
                i1,j1 = InBox[k1]
                I = i0+i1
                J = j0+j1
                
                box1 = f(I,J,i)
                
                for k2 in range(k1+1,n):
                    
                    i2,j2=InBox[k2]
                    I1 = i0+i2
                    J1 = j0+j2
                    
                    box2 = f(I1,J1,i)
                    Q[(box1,box2)]  = 2*c
            

In [6]:
# Submit for solution
answer = sampler.sample_qubo(Q)
R = answer.first[0]

Input the solution to a matrix

In [7]:
D = list()
print('Solution')
for i in range(n):
    D_=list()
    for j in range(n):
        for k in range(n):
            if R.get(f(i,j,k))==1:
                print(k+1,end=' ')
                D_.append(k+1)
                break
    D.append(D_)
    print()

Solution
2 9 5 3 6 4 1 7 8 
8 4 1 9 5 7 6 2 3 
3 7 6 1 8 2 5 9 4 
4 8 7 6 9 5 3 1 2 
9 1 2 8 7 3 4 5 6 
6 5 3 2 4 1 9 8 7 
7 6 9 5 3 8 2 4 1 
5 2 4 7 1 6 8 3 9 
1 3 8 4 2 9 7 6 5 


Check if the solution found is a solution for the given problem.

In [8]:
flag=True
for i in range(n):
    for j in range(n):
        if S[i][j]!=0 and S[i][j]!=D[i][j]:
            print('At ({},{}) the solution does not match the question'.format(i,j))
            flag=False
if flag:
    print('The solution found is a solution for the given question')

The solution found is a solution for the given question


Check if the solution found is a valid Sudoku solution.

In [15]:
flag=True
for i in rn:
    for j in rn:
        x=set()
        y=set()
        for k in rn:
            x.add(D[j][k])
            y.add(D[k][j])
        if len(x)!=n:
            flag=False
            print('In {}th row, only {} numbers are present.'.format(j,x))
        if len(x)!=n:
            flag=False
            print('In {}th column, only {} numbers are present.'.format(j,y))
        for k in rn:
            
            i0 = (j // N)*N
            j0 = (k // N)*N
            x=set()
            # Boxes
            for k1 in rn:
                i1,j1 = InBox[k1]
                I = i0+i1
                J = j0+j1
                x.add(D[I][J])
            if len(x)!=n:
                flag=False
                print('In the box ({},{}), only {} numbers are present.'.format(I,J,x))
if flag:
    print('Solution is a valid sudoku')

Solution is a valid sudoku
